# Analyze results from Leon French's java app

A split-half training set and a split-quarter training set were each run through Leon's java app on jjm15. Later we fed Leon's app full matrices under a couple of different contexts. It re-ranked probes on every iteration. 


## Data loading and organization

The first block does not need to be run, but it can without a problem. It contains old lists of bad or cancelled optimization runs. Typically this would be considered code bloat and deleted, but these are kept intentionally as this block is the best record of their existence, how long their computation took, etc. These data are typically collected by manually copy-pasting from stdout when executing runs, scraping from log files, etc. and can be moderately time-consuming. There is value to these data in some analyses of performance that aren't relevant for publication, but are for understanding the code used.

In [1]:
""" Ignorable bad data, stored in a way that will execute quickly without corrupting analyses.
    Do not delete; keep around for record-keeping.
"""

base_dir = "/home/mike/projects/bams_and_allen"

# These runs were killed, some because they were run with too few CPUs or insufficient RAM and we gave up.
# We do nothing with them here, but this can help with cross-referencing files in the filesystem.
java_cancels = [
    {'subdir': "whole_23", 'filename': "LOOResults.1582223481773.txt", },
    {'subdir': "whole_23", 'filename': "LOOResults.1582232409921.txt", },
    {'subdir': "whole_23", 'filename': "LOOResults.1582428218753.txt", },
    {'subdir': "whole", 'filename': "LOOResults.1582223481773.txt", },
]

# These runs are either ignored or cancelled because they were fed a bad connectivity matrix.
java_diluted_results = [
    {   # 2020-03-18 - Running on jjm14 - should finish around Friday 3/20 mid-day
        'name': 'java_whole_23c_60g', 'plot': False,  # identical to 12c, n/a yet
        'base_dir': base_dir, 'subdir': "whole_23c_60g", 'filename': "LOOResults.1583164067177.txt", 
        'cores': 23, 'ram': 60, 'columns': 1139, 'genes': 15745, 'first_estimates': [397, 432, 450, 437, ],
        'command': "java -Xmx60g -cp ./BAMSandAllen_fat.jar ubic.BAMSandAllen.MatrixPairs.FromFileMatrixPair 23 /home/mikes/projects/bams_allen_java/whole_23c_60g/hcp_conn_sim_whole.tsv /home/mikes/projects/bams_allen_java/whole_23c_60g/ahba_expr_whole.tsv",
        'expr': "/home/mikes/projects/bams_allen_java/whole_23c_60g/ahba_expr_whole.tsv",
        'conn': "/home/mikes/projects/bams_allen_java/whole_23c_60g/hcp_conn_sim_whole.tsv",
        'start': "2020-03-02 10:42:11", 'node': "jjm14",
    },
    {
        'name': 'java_whole_12c_10g', 'plot': True,
        'base_dir': base_dir, 'subdir': "whole", 'filename': "LOOResults.1582232409921.txt", 
        'cores': 12, 'ram': 10, 'columns': 1139, 'genes': 15745, 'first_estimates': [537, 511, 467, 533, ],
        'command': "java -Xmx10g -cp ./BAMSandAllen_fat.jar ubic.BAMSandAllen.MatrixPairs.FromFileMatrixPair 12 /home/mikes/projects/bams_allen_java/whole/hcp_conn_sim_whole.tsv /home/mikes/projects/bams_allen_java/whole/ahba_expr_whole.tsv",
        'expr': "/home/mikes/projects/bams_allen_java/whole/ahba_expr_whole.tsv",
        'conn': "/home/mikes/projects/bams_allen_java/whole/hcp_conn_sim_whole.tsv",
        'start': "2020-02-20 16:04:33", 'end': "2020-03-13 05:35:49", 'node': "jjm15",
    },
    {
        'name': 'java_half_12c_10g', 'plot': True,
        'base_dir': base_dir, 'subdir': "202", 'filename': "LOOResults.1580525004509.txt",
        'columns': 568, 'genes': 15745,
        'expr': "/home/mikes/projects/bams_allen_java/202/ahba_expr_train202.tsv",
        'conn': "/home/mikes/projects/bams_allen_java/202/hcp_conn_sim_train202.tsv",
        'start': "2020-01-31 21:44:32", 'end': "2020-02-08 12:40:30",
    },
    {
        'name': 'java_quarter_23c_60g_1', 'plot': False,  # identical to 12c and to _2
        'base_dir': base_dir, 'subdir': "quarter_23_60", 'filename': "LOOResults.1583035707345.txt", 
        'cores': 23, 'ram': 60, 'columns': 283, 'genes': 15745, 'first_estimates': [26, 21, 26, 26, 21, ],
        'command': "java -Xmx60g -cp ./BAMSandAllen_fat.jar ubic.BAMSandAllen.MatrixPairs.FromFileMatrixPair 23 /home/mikes/projects/bams_allen_java/quarter_23_60/hcp_conn_sim_train402.tsv /home/mikes/projects/bams_allen_java/quarter_23_60/ahba_expr_train402.tsv",
        'expr': "/home/mikes/projects/bams_allen_java/quarter_23_60/ahba_expr_train402.tsv",
        'conn': "/home/mikes/projects/bams_allen_java/quarter_23_60/hcp_conn_sim_train402.tsv",
        'start': "2020-02-29 23:08:45", 'end': "2020-03-02 03:22:16", 'node': "jjm14",
    },
    {
        'name': 'java_quarter_23c_60g_2', 'plot': False,  # identical to 12c and to _1
        'base_dir': base_dir, 'subdir': "402", 'filename': "LOOResults.1582340405040.txt",
        'columns': 283, 'genes': 15745,
        'start': "2020-02-21 22:01:02", 'end': "2020-02-23 07:15:15",
    },
    {
        'name': 'java_quarter_12c_10g', 'plot': True,
        'base_dir': base_dir, 'subdir': "402", 'filename': "LOOResults.1580573710986.txt",
        'columns': 283, 'genes': 15745,
        'start': "2020-02-01 11:15:39", 'end': "2020-02-04 04:00:30",
    },
    {
        'name': 'java_parcels_23c_60g', 'plot': True,
        'base_dir': base_dir, 'subdir': "whole_glasser_23c_60g", 'filename': "LOOResults.1584153479004.txt",
        'columns': 177, 'genes': 15745,
        'start': "2020-03-13 22:37:49", 'end': "2020-03-14 07:50:56",
    },
]

pygest_diluted_results = [
    {   # 2020-03-18 - running on jjm15
        'name': 'pygest_whole_evry_12c', 'plot': False,  # n/a yet
        'base_dir': base_dir, 'subdir': 'pygest_whole_evry_12c',
        'filename': "",
        'columns': 1139, 'genes': 15745, 'cores': 12,
        'node': "jjm15",
    },
    {
        'name': 'pygest_whole_smrt_12c', 'plot': False,  # identical to 4c
        'base_dir': base_dir, 'subdir': 'pygest_whole_smrt_12c',
        'filename': "ahbaexprwholedf_comp-hcpniftismoothgrandmeansim_mask-none_norm-srs_adj-none.tsv",
        'columns': 1139, 'genes': 15745, 'cores': 12,
        'expr': "/data/cache/ahba_expr_whole.df",
        'conn': "/data/conn/hcp_niftismooth_grandmean_sim.df",
        'start': "2020-03-16 23:59:05", 'end': "2020-03-17 06:27:26", 'node': "jjm15",
    },
    {
        'name': 'pygest_whole_smrt_4c', 'plot': True,
        'base_dir': base_dir, 'subdir': 'pygest_whole_smrt_4c',
        'filename': "ahbaexprwholedf_comp-hcpniftismoothgrandmeansim_mask-none_norm-srs_adj-none.tsv",
        'columns': 1139, 'genes': 15745, 'cores': 4,
        'expr': "/data/cache/ahba_expr_whole.df",
        'conn': "/data/conn/hcp_niftismooth_grandmean_sim.df",
        'start': "2020-03-17 11:34:12", 'end': "2020-03-18 01:33:25", 'node': "jjm15",
    },
    {
        'name': 'pygest_whole_once_8c', 'plot': True,
        'base_dir': base_dir, 'subdir': 'pygest_whole_once_8c',
        'filename': "ahbaexprwholedf_comp-hcpniftismoothgrandmeansim_mask-none_norm-srs_adj-none.tsv",
        'columns': 1139, 'genes': 15745, 'cores': 8,
        'start': "2020-03-17 11:36:18", 'end': "2020-03-17 22:44:55", 'node': "jjm15",
    },
    {   # 2020-03-18 - running on jjm6
        'name': 'pygest_half_evry_6c', 'plot': False,  # n/a yet
        'base_dir': base_dir, 'subdir': 'pygest_half_evry_6c',
        'filename': "",
        'columns': 568, 'genes': 15745, 'cores': 6,
        'node': "jjm6",
    },
    {
        'name': 'pygest_half_smrt_4c', 'plot': False,  # old version, upgraded to 6c
        'base_dir': base_dir, 'subdir': 'parby-wellid_splby-wellid_batch-train00202/tgt-max_algo-smrt',
        'filename': "sub-all_comp-hcpniftismoothgrandmeansim_mask-none_norm-srs_adj-none.tsv",
        'columns': 568, 'genes': 15745, 'cores': 4,
        'start': "2019-12-28 13:43:11", 'end': "2019-12-28 16:21:22", 'node': "jjm15",
    },
    {
        'name': 'pygest_half_smrt_6c', 'plot': True,
        'base_dir': base_dir, 'subdir': 'pygest_half_smrt_6c',
        'filename': "sub-all_comp-hcpniftismoothgrandmeansim_mask-none_norm-srs_adj-none.tsv",
        'columns': 568, 'genes': 15745, 'cores': 6,
        'start': "2020-03-17 18:48:25", 'end': "2020-03-17 21:30:57", 'node': "jjm6",
    },
    {   # 2020-03-18 - running on jjm12
        'name': 'pygest_quarter_evry_12c', 'plot': False,  # n/a yet
        'base_dir': base_dir, 'subdir': 'pygest_quarter_evry_12c',
        'filename': "",
        'columns': 283, 'genes': 15745, 'cores': 12,
        'node': "jjm12",
    },
    {
        'name': 'pygest_quarter_smrt_4c', 'plot': True,
        'base_dir': base_dir, 'subdir': 'parby-wellid_splby-wellid_batch-train00402/tgt-max_algo-smrt',
        'filename': "sub-all_comp-hcpniftismoothgrandmeansim_mask-none_norm-srs_adj-none.tsv",
        'columns': 283, 'genes': 15745, 'cores': 4,
        'start': "2020-01-31 14:16:40", 'end': "2020-01-31 15:24:19", 'node': "jjm8",
    },
    {   # 2020-03-18 - running on jjm5, killed and resumed with 14 cores
        'name': 'pygest_glasser_evry_8c', 'plot': False,  # n/a yet
        'base_dir': base_dir, 'subdir': 'pygest_glasser_evry_8c',
        'filename': "",
        'columns': 177, 'genes': 15745, 'cores': 8,
        'node': "jjm5",
    },
    {
        'name': 'pygest_glasser_smrt_8c', 'plot': True,
        'base_dir': base_dir, 'subdir': 'pygest_glasser_smrt_8c',
        'filename': "ahbaexprwholeparbyglasserdf_comp-hcpconnsimwholeparbyglasser_mask-none_norm-none_adj-none.tsv",
        'columns': 177, 'genes': 15745, 'cores': 8,
        'start': "2020-03-17 21:09:35", 'end': "2020-03-17 21:40:18", 'node': "jjm5",
    },
]


In [1]:
""" Set up the context. 

    Good data, used for analyses
"""

import os


base_dir = "/home/mike/projects/bams_and_allen"

template_for_copy_paste_without_data = [
    {
        'name': "", 'base_dir': base_dir, 'subdir': "",
        'filename': "",
        'columns': 0, 'genes': 15745, 'cores': 0, 'first_estimate': 0,
        'expr': "", 'conn': "",
        'start': "", 'end': "", 'node': "", 'plot': True,
    },
]

java_results = [
    {
        'name': "java_whole_23c_60g", 'base_dir': base_dir, 'subdir': "java/whole_23_60",
        'filename': "LOOResults.1584709233635.txt",
        'columns': 1139, 'genes': 15745, 'cores': 23, 'first_estimate': 454,
        'expr': "ahba_expr_whole.tsv", 'conn': "hcp_conn_sim_whole.tsv",
        'start': "2020-03-20 08:54:52", 'end': "", 'node': "jjm14", 'plot': True,
    },
    {
        'name': "java_half_23c_60g", 'base_dir': base_dir, 'subdir': "java/half_23_60",
        'filename': "LOOResults.1584762903665.txt",
        'columns': 568, 'genes': 15745, 'cores': 23, 'first_estimate': 113,
        'expr': "ahba_expr_train202.tsv", 'conn': "hcp_conn_sim_train202.tsv",
        'start': "2020-03-20 23:53:30", 'end': "2020-03-24 21:24:19", 'node': "jjm15", 'plot': True,
    },
    {
        'name': "java_quarter_23c_60g", 'base_dir': base_dir, 'subdir': "java/quarter_23_60",
        'filename': "LOOResults.1584675748302.txt",
        'columns': 283, 'genes': 15745, 'cores': 23, 'first_estimate': 26,
        'expr': "ahba_expr_train402.tsv", 'conn': "hcp_conn_sim_train402.tsv",
        'start': "2020-03-19 23:42:07", 'end': "2020-03-20 23:47:04", 'node': "jjm15", 'plot': True,
    },
    {
        'name': "java_glasser_23c_60g", 'base_dir': base_dir, 'subdir': "java/glasser_23_60",
        'filename': "LOOResults.1584675847767.txt",
        'columns': 176, 'genes': 15745, 'cores': 23, 'first_estimate': 0,
        'expr': "ahba_expr_whole_parby-glasser.tsv", 'conn': "hcp_conn_sim_whole_parby-glasser.tsv",
        'start': "2020-03-19 23:43:59", 'end': "2020-03-20 08:50:29", 'node': "jjm14", 'plot': True,
    },
]

pygest_results = [
    {
        'name': "pygest_whole_8_smrt", 'base_dir': base_dir, 'subdir': "pygest/whole_8_smrt",
        'filename': "ahbaexprwholedf_comp-hcpconnsimwhole_mask-none_norm-none_adj-none.tsv",
        'columns': 1139, 'genes': 15745, 'cores': 8, 'first_estimate': 0,
        'expr': "ahba_expr_whole.df", 'conn': "hcp_conn_sim_whole.df",
        'start': "2020-03-19 23:52:24", 'end': "2020-03-20 07:59:02", 'node': "jjm6", 'plot': True,
    },
    {
        'name': "pygest_whole_4_once", 'base_dir': base_dir, 'subdir': "pygest/whole_4_once",
        'filename': "ahbaexprwholedf_comp-hcpconnsimwhole_mask-none_norm-none_adj-none.tsv",
        'columns': 1139, 'genes': 15745, 'cores': 4, 'first_estimate': 0,
        'expr': "ahba_expr_whole.df", 'conn': "hcp_conn_sim_whole.df",
        'start': "2020-03-20 08:39:27", 'end': "2020-03-20 15:55:06", 'node': "jjm7", 'plot': True,
    },
    {
        'name': "pygest_whole_8_evry", 'base_dir': base_dir, 'subdir': "pygest/whole_8_evry",
        'filename': "logfile.txt.grepped.as.tsv",
        'columns': 1139, 'genes': 15745, 'cores': 8, 'first_estimate': 0,
        'expr': "ahba_expr_whole.df", 'conn': "hcp_conn_sim_whole.df",
        'start': "", 'end': "", 'node': "", 'plot': True,
    },
    {
        'name': "pygest_half_8_smrt", 'base_dir': base_dir, 'subdir': "pygest/half_8_smrt",
        'filename': "ahbaexprtrain202df_comp-hcpconnsimtrain202_mask-none_norm-none_adj-none.tsv",
        'columns': 568, 'genes': 15745, 'cores': 8, 'first_estimate': 0,
        'expr': "ahba_expr_train202.df", 'conn': "hcp_conn_sim_train202.df",
        'start': "2020-03-19 23:51:36", 'end': "2020-03-20 02:39:16", 'node': "jjm6", 'plot': True,
    },
    {
        'name': "pygest_half_4_once", 'base_dir': base_dir, 'subdir': "pygest/half_4_once",
        'filename': "ahbaexprtrain202df_comp-hcpconnsimtrain202_mask-none_norm-none_adj-none.tsv",
        'columns': 568, 'genes': 15745, 'cores': 4, 'first_estimate': 0,
        'expr': "ahba_expr_train202.df", 'conn': "hcp_conn_sim_train202.df",
        'start': "2020-03-20 08:16:08", 'end': "2020-03-20 10:30:12", 'node': "jjm7", 'plot': True,
    },
    {
        'name': "pygest_half_8_evry", 'base_dir': base_dir, 'subdir': "pygest/half_8_evry",
        'filename': "logfile.txt.grepped.as.tsv",
        'columns': 568, 'genes': 15745, 'cores': 8, 'first_estimate': 0,
        'expr': "ahba_expr_train202.df", 'conn': "hcp_conn_sim_train202.df",
        'start': "", 'end': "", 'node': "", 'plot': True,
    },
    {
        'name': 'pygest_quarter_8_smrt', 'base_dir': base_dir, 'subdir': 'pygest/quarter_8_smrt',
        'filename': "ahbaexprtrain402df_comp-hcpconnsimtrain402_mask-none_norm-none_adj-none.tsv",
        'columns': 283, 'genes': 15745, 'cores': 8, 'first_estimate': 0,
        'expr': "ahba_expr_train402.df", 'conn': "hcp_conn_sim_train402.df",
        'start': "2020-03-19 23:49:25", 'end': "2020-03-20 00:29:21", 'node': "jjm5", 'plot': True,
    },    
    {
        'name': 'pygest_quarter_15_smrt', 'base_dir': base_dir, 'subdir': 'pygest/quarter_15_smrt_v114',
        'filename': "ahbaexprtrain402df_comp-hcpconnsimtrain402_mask-none_norm-none_adj-none.tsv",
        'columns': 283, 'genes': 15745, 'cores': 15, 'first_estimate': 0,
        'expr': "ahba_expr_train402.df", 'conn': "hcp_conn_sim_train402.df",
        'start': "2020-03-21 19:25:58", 'end': "2020-03-21 20:05:14", 'node': "jjm6", 'plot': True,
    },    
    {
        'name': 'pygest_quarter_4_once', 'base_dir': base_dir, 'subdir': 'pygest/quarter_4_once',
        'filename': "ahbaexprtrain402df_comp-hcpconnsimtrain402_mask-none_norm-none_adj-none.tsv",
        'columns': 283, 'genes': 15745, 'cores': 4, 'first_estimate': 0,
        'expr': "ahba_expr_train402.df", 'conn': "hcp_conn_sim_train402.df",
        'start': "2020-03-20 08:15:28", 'end': "2020-03-20 09:02:56", 'node': "jjm7", 'plot': True,
    },
    {
        'name': 'pygest_quarter_8_evry', 'base_dir': base_dir, 'subdir': 'pygest/quarter_8_evry',
        'filename': "logfile.txt.grepped.as.tsv",
        'columns': 283, 'genes': 15745, 'cores': 8, 'first_estimate': 0,
        'start': "", 'end': "", 'node': "", 'plot': True,
    },    
    {
        'name': 'pygest_glasser_8_smrt', 'base_dir': base_dir, 'subdir': 'pygest/glasser_8_smrt',
        'filename': "ahbaexprwholeparbyglasserdf_comp-hcpconnsimwholeparbyglasser_mask-none_norm-none_adj-none.tsv",
        'columns': 176, 'genes': 15745, 'cores': 8, 'first_estimate': 0,
        'expr': "ahba_expr_whole_parby-glasser.df", 'conn': "hcp_conn_sim_whole_parby-glasser.df",
        'start': "2020-03-19 23:45:43", 'end': "2020-03-20 00:07:44", 'node': "jjm5", 'plot': True,
    },
    {
        'name': 'pygest_glasser_4_once', 'base_dir': base_dir, 'subdir': 'pygest/glasser_4_once',
        'filename': "ahbaexprwholeparbyglasserdf_comp-hcpconnsimwholeparbyglasser_mask-none_norm-none_adj-none.tsv",
        'columns': 176, 'genes': 15745, 'cores': 4, 'first_estimate': 0,
        'expr': "ahba_expr_whole_parby-glasser.df", 'conn': "hcp_conn_sim_whole_parby-glasser.df",
        'start': "2020-03-20 08:14:32", 'end': "2020-03-20 08:38:04", 'node': "jjm7", 'plot': True,
    },
    {
        'name': 'pygest_glasser_12_smrt', 'base_dir': base_dir, 'subdir': 'pygest/glasser_12_smrt',
        'filename': "ahbaexprwholeparbyglasserdf_comp-hcpconnsimwholeparbyglasser_mask-none_norm-none_adj-none.tsv",
        'columns': 176, 'genes': 15745, 'cores': 12, 'first_estimate': 0,
        'expr': "ahba_expr_whole_parby-glasser.df", 'conn': "hcp_conn_sim_whole_parby-glasser.df",
        'start': "2020-03-20 08:48:36", 'end': "2020-03-20 09:06:33", 'node': "jjm6", 'plot': True,
    },
    {
        'name': 'pygest_glasser_12_once', 'base_dir': base_dir, 'subdir': 'pygest/glasser_12_once',
        'filename': "ahbaexprwholeparbyglasserdf_comp-hcpconnsimwholeparbyglasser_mask-none_norm-none_adj-none.tsv",
        'columns': 176, 'genes': 15745, 'cores': 12, 'first_estimate': 0,
        'expr': "ahba_expr_whole_parby-glasser.df", 'conn': "hcp_conn_sim_whole_parby-glasser.df",
        'start': "2020-03-20 08:46:04", 'end': "2020-03-20 09:02:46", 'node': "jjm5", 'plot': True,
    },
    {
        'name': 'pygest_glasser_8_evry', 'base_dir': base_dir, 'subdir': 'pygest/glasser_8_evry',
        'filename': "logfile.txt.grepped.as.tsv",
        'columns': 176, 'genes': 15745, 'cores': 8, 'first_estimate': 0,
        'expr': "ahba_expr_whole_parby-glasser.df", 'conn': "hcp_conn_sim_whole_parby-glasser.df",
        'start': "", 'end': "", 'node': "jjm5", 'plot': True,
    },
    {
        'name': 'pygest_quarter_02_smrt', 'base_dir': base_dir, 'subdir': 'pygest/quarter_02_smrt',
        'filename': "ahbaexprtrain40202df_comp-hcpconnsimtrain40202_mask-none_norm-none_adj-none.tsv",
        'columns': 283, 'genes': 15745, 'cores': 2, 'first_estimate': 0,
        'expr': "ahba_expr_train40202.df", 'conn': "hcp_conn_sim_train40202.df",
        'start': "2020-03-24 12:15:59", 'end': "2020-03-24 13:26:09", 'node': "jjm6", 'plot': True,
    },    
    {
        'name': 'pygest_quarter_10_smrt', 'base_dir': base_dir, 'subdir': 'pygest/quarter_10_smrt',
        'filename': "ahbaexprtrain40210df_comp-hcpconnsimtrain40210_mask-none_norm-none_adj-none.tsv",
        'columns': 283, 'genes': 15745, 'cores': 10, 'first_estimate': 0,
        'expr': "ahba_expr_train40210.df", 'conn': "hcp_conn_sim_train40210.df",
        'start': "2020-03-24 12:15:21", 'end': "2020-03-24 12:56:45", 'node': "jjm6", 'plot': True,
    },    
]

all_results = java_results + pygest_results


In [2]:
""" Read the results files """

import pandas as pd
from datetime import datetime


time_format = "%Y-%m-%d %H:%M:%S"

def peak_index(df):
    """ Find the index at the peak (not quite the same as the index with the highest Mantel) """
    
    last_r = -1.0
    last_idx = 0
    for idx, row in df.sort_index(ascending=True).iterrows():
        if row['r'] < last_r:
            print("    current ({}:{:0.6f}) < prior ({}:{:0.6f})".format(idx, row['r'], last_idx, last_r))
            return last_idx
        last_r = row['r']
        last_idx = idx
    # The for loop should never complete, but just in case...
    return 0


# Read each result file, adjusting both result types to match
for rslt in all_results:
    if os.path.isfile(os.path.join(rslt['base_dir'], rslt['subdir'], rslt['filename'])):
        print("{name:<24.24} {subdir:<42.42} {filename:<42.42} - {check}".format(**rslt, check=u'\u2713'))
        rslt['df'] = pd.read_csv(
            os.path.join(rslt['base_dir'], rslt['subdir'], rslt['filename']),
            sep="," if rslt['name'][:4] == "java" else "\t",
            index_col=0,
            header=None if rslt['name'][:4] == "java" else 0,
        )
        if rslt['name'][:4] == "java":
            rslt['df'].columns = ['r', 'probe_id', ]
        if rslt['name'][:6] == "pygest":
            rslt['df'] = rslt['df'][['r', 'probe_id', ]]

        # Both have an index representing order of importance; but pygest is 1-based and java is 0-based.
        # Make them comparable, then name the index
        rslt['df'] = rslt['df'].sort_index(ascending=True).reset_index(drop=True)[['probe_id', 'r', ]]
        rslt['df'].index.name = 'seq'

        # Calculate elapsed time if start and end times are available
        if 'end' in rslt.keys() and 'start' in rslt.keys() and rslt['end'] != "" and rslt['start'] != "":
            elapsed = datetime.strptime(rslt['end'], time_format) - datetime.strptime(rslt['start'], time_format)
            rslt['elapsed_hours'] = elapsed.total_seconds() / 3600
            # print("  {} to {} : {} == {:0.1f} hrs".format(rslt['start'], rslt['end'], elapsed, elapsed.total_seconds() / 3600))
            # print(rslt['df'].sample(2))
        else:
            rslt['elapsed_hours'] = 0.0

        # Calculate the index position at which the highest correlation was achieved.
        rslt['peak_idx'] = peak_index(rslt['df'])

        print("  : {shape} : took {elapsed_hours:0.1f} hrs, peak @{peak_idx}".format(
            **rslt, shape=rslt['df'].shape
        ))
    else:
        print("{name:<24.24} {subdir:<42.42} {filename:<42.42} - missing".format(**rslt))



java_whole_23c_60g       java/whole_23_60                           LOOResults.1584709233635.txt               - ✓
  : (2100, 2) : took 0.0 hrs, peak @0
java_half_23c_60g        java/half_23_60                            LOOResults.1584762903665.txt               - ✓
    current (15648:0.403949) < prior (15647:0.403952)
  : (15745, 2) : took 93.5 hrs, peak @15647
java_quarter_23c_60g     java/quarter_23_60                         LOOResults.1584675748302.txt               - ✓
    current (15628:0.450709) < prior (15627:0.450712)
  : (15745, 2) : took 24.1 hrs, peak @15627
java_glasser_23c_60g     java/glasser_23_60                         LOOResults.1584675847767.txt               - ✓
    current (15735:0.472444) < prior (15734:0.472472)
  : (15745, 2) : took 9.1 hrs, peak @15734
pygest_whole_8_smrt      pygest/whole_8_smrt                        ahbaexprwholedf_comp-hcpconnsimwhole_mask- - ✓
    current (15649:0.410068) < prior (15648:0.410074)
  : (15745, 2) : took 8.1 hrs, peak @156

In [3]:
""" Combine all results into a single dataframe. """

# This should not be done in the first block because items are added as results are read directly above.
df_all_results = pd.DataFrame(all_results).set_index('name')


In [4]:
""" Combine all ranks into a single dataframe with probe_ids as the index. """

def swap_index_for_probe(df, column_name):
    return df.reset_index()[['probe_id', 'seq']].set_index('probe_id').sort_values('seq', ascending=True).rename(columns={'seq': column_name})

df_all_orders = pd.concat([
    swap_index_for_probe(result['df'], result['name']) for result in all_results if 'df' in result.keys()
], axis=1)
df_all_orders['mean_seq'] = df_all_orders.dropna(axis=1).mean(axis=1)
df_all_orders = df_all_orders.sort_values('mean_seq')

# The index of df_all_orders is now all probe_ids in average order of removal.
df_all_orders['mean_seq']


probe_id
1053004       26.0000
1055653       30.6250
1024320       35.5625
1016265       36.2500
1031623       37.0625
              ...    
1017936    15551.1875
1066109    15599.0625
1040880    15622.1250
1016934    15625.0000
1031901    15635.5625
Name: mean_seq, Length: 15745, dtype: float64

## Data visualization

In [5]:
""" Plot each result against each other result, by seq in order of the averaged probe list. """

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kendalltau
import numpy as np


def plot_ranks(df, x, x_peak, y, y_peak):
    """ multi-scatter-plot to compare gene rankings from two sources """
    
    # Make x the ruler, in its own order, and plot y based on x's probe-order
    num_complete = min(np.isfinite(df[y].values).sum(), np.isfinite(df[x].values).sum())
    plottable_df = df[[y, x]].dropna(axis='index', how='any')
    plottable_df = plottable_df.sort_values(y)
    # import pdb; pdb.set_trace()
    plottable_df['y_hi'] = plottable_df[y] >= y_peak
    plottable_df = plottable_df.sort_values(x)
    plottable_df['x_hi'] = plottable_df[x] >= x_peak
    
    fig, axes = plt.subplots(figsize=(6,6))
    
    r, p = kendalltau(df[y], df[x])
    r2, p2 = kendalltau(plottable_df[y], plottable_df[x])
    print("{:<24} vs {:<24} : Kendall tau = {:0.3f}, p={:0.4f} (dropna {:0.3f} p={:0.4f})".format(
        y, x, r, p, r2, p2
    ))
    sns.regplot(
        data=plottable_df, y=y, x=x, color='lightskyblue', scatter_kws={'s': 2}, ax=axes,
    )

    if len(plottable_df) == len(df):
        sns.scatterplot(data=plottable_df[plottable_df['x_hi'] | plottable_df['y_hi']],
                        y=y, x=x, color='dodgerblue', size=2)
        sns.scatterplot(data=plottable_df[plottable_df['x_hi'] & plottable_df['y_hi']],
                        y=y, x=x, color='midnightblue', size=5)
        axes.axhline(y_peak, color='red', linestyle=":")
        axes.axvline(x_peak, color='red', linestyle=":")
        axes.set_xlim(-100, 16000)
        axes.set_ylim(-100, 16000)
        axes.set_xticks([0, 5000, 10000, x_peak])
        axes.set_yticks([0, 5000, 10000, y_peak])
        axes.get_legend().remove()
        fig.suptitle("Kendall tau = {:0.3f}, p={:0.5f}\n{} past x, {} past y, {} agree".format(
            r, p, sum(plottable_df['x_hi']), sum(plottable_df['y_hi']),
            sum(plottable_df['y_hi'] & plottable_df['x_hi'])
        ))
    else:
        axes.set_xlim(-100, num_complete + 200)
        axes.set_ylim(-100, num_complete + 200)
        axes.set_xticks([0, num_complete])
        axes.set_yticks([0, num_complete])
        fig.suptitle("Kendall tau = {:0.3f}, p={:0.5f}\nonly {:,} complete".format(
            r2, p2, num_complete
        ))
        
    return fig, axes


In [8]:
""" Plot multiple runs juxtaposed with each other. """

import matplotlib.pyplot as plt
import seaborn as sns


def plot_runs(dfs):
    """ plot the rising Mantel r values for optimization runs. """
    
    fig, axes = plt.subplots(nrows=2, figsize=(6, 6))
    
    plottable_df = pd.concat([
        d['df'][['r']].rename(columns={'r': d['name']}) for d in dfs
    ], axis=1)

    sns.lineplot(data=plottable_df[-1200:].sort_index(), ax=axes[0])
    axes[0].ylabel = "Mantel r"
    sns.lineplot(data=plottable_df.sort_index(), ax=axes[1])
    axes[1].ylabel = "Mantel r"
    
    axes[0].get_legend().remove()
    
    return fig, axes


In [9]:
""" Execute both plots for each comparison pair """

valid_columns = [c for c in df_all_orders.columns if c != 'mean_seq' and df_all_results.loc[c, 'plot']]
for j, y in enumerate(valid_columns):
    for i, x in enumerate(valid_columns):
        if y != x:
            x_peak = df_all_results.loc[x, 'peak_idx']
            y_peak = df_all_results.loc[y, 'peak_idx']
            
            f, a = plot_ranks(df_all_orders, x, x_peak, y, y_peak)
            f.savefig("order_comparisons/{}-vs-{}.png".format(y, x))
            plt.close(f)
            
            f, a = plot_runs([{'name': name, 'df': df_all_results.loc[name, 'df']} for name in [y, x]])
            f.savefig("order_comparisons/{}-and-{}.png".format(y, x))
            plt.close(f)



java_whole_23c_60g       vs java_half_23c_60g        : Kendall tau = nan, p=nan (dropna 0.467 p=0.0000)
java_whole_23c_60g       vs java_quarter_23c_60g     : Kendall tau = nan, p=nan (dropna 0.382 p=0.0000)
java_whole_23c_60g       vs java_glasser_23c_60g     : Kendall tau = nan, p=nan (dropna 0.317 p=0.0000)
java_whole_23c_60g       vs pygest_whole_8_smrt      : Kendall tau = nan, p=nan (dropna 0.740 p=0.0000)
java_whole_23c_60g       vs pygest_whole_4_once      : Kendall tau = nan, p=nan (dropna 0.740 p=0.0000)
java_whole_23c_60g       vs pygest_whole_8_evry      : Kendall tau = nan, p=nan (dropna 1.000 p=0.0000)
java_whole_23c_60g       vs pygest_half_8_smrt       : Kendall tau = nan, p=nan (dropna 0.391 p=0.0000)
java_whole_23c_60g       vs pygest_half_4_once       : Kendall tau = nan, p=nan (dropna 0.378 p=0.0000)
java_whole_23c_60g       vs pygest_half_8_evry       : Kendall tau = nan, p=nan (dropna 0.528 p=0.0000)
java_whole_23c_60g       vs pygest_quarter_8_smrt    : Kendall t

### Create a few custom plots to support the narrative and complement auto-build plots from above.

In [10]:
""" Create the following plots with custom lists of results to include. """

for png_name, plottable in [
    (
        "java_by_expr.png",
        [{'name': name, 'df': df_all_results.loc[name, 'df']} for name in [
            'java_whole_23c_60g', 'java_half_23c_60g', 'java_quarter_23c_60g', 'java_glasser_23c_60g',
        ]],
    ),
    (
        "pygest_by_expr.png",
        [{'name': name, 'df': df_all_results.loc[name, 'df']} for name in [
            'pygest_whole_8_smrt', 'pygest_half_8_smrt', 'pygest_quarter_8_smrt', 'pygest_glasser_8_smrt',
        ]],
    ),
    (
        "glassers_all.png",
        [{'name': name, 'df': df_all_results.loc[name, 'df']} for name in [
            'pygest_glasser_8_smrt', 'pygest_glasser_4_once', 'pygest_whole_8_smrt', 'java_glasser_23c_60g', 'java_whole_23c_60g',
        ]],
    ),
    (
        "once_trio.png",
        [{'name': name, 'df': df_all_results.loc[name, 'df']} for name in [
            'pygest_whole_4_once', 'pygest_whole_8_smrt', 'java_whole_23c_60g',
        ]],
    ),
]:
    print("Plotting {}...".format(png_name))
    f, a = plot_runs(plottable)
    f.savefig("order_comparisons/{}".format(png_name))
    plt.close(f)
    

Plotting java_by_expr.png...
Plotting pygest_by_expr.png...
Plotting glassers_all.png...
Plotting once_trio.png...


----

## Old, experimental, and backup code

----

In [10]:
""" If necessary, build a toy dataframe for experimentation. """

littledf = pd.DataFrame(
    data={
        'probe_id': [3456, 6543, 4567, 5678, 1111, 9876, ],
        'r': [0.022, 0.031, 0.050, 0.100, 0.111, 0.055, ]
    },
    index=[0, 1, 2, 3, 4, 5, ],
)
littledf.index.name = 'seq'
littledf


,probe_id,r
seq,,
0,3456,0.022
1,6543,0.031
2,4567,0.050
3,5678,0.100
4,1111,0.111
5,9876,0.055


In [11]:
""" Tweak Leon's results to match our format.
    - He ranks 0-based; we rank 1-based.
    - He gives the final probes the same ranking and r-value; we rank them sequentially with r=0.00.
    - He orders from 0-end ascending; we order descending from 15745 down to 1.
"""

def format_leons_as_mikes(df):
    df.columns = ['seq', 'r', 'probe_id']
    df = df.sort_values('seq')
    df['seq'] = range(1, len(df) + 1)
    return df
    
# results_df_from_split_half_java = format_leons_as_mikes(results_df_from_split_half_java)
# results_df_from_split_quarter_java = format_leons_as_mikes(results_df_from_split_quarter_java)
